# Implementing Fraud Detection using Strands Agents & Databricks

This notebook demonstrates how to build a fraud detection system using Strands Agents on Databricks

### Why Transition to Strands SDK?
AWS Strands SDK offers enhanced capabilities for building truly autonomous agents:

- Model-driven approach
- Built-in workflow & tool orchestration
- Specialized agent roles
- Structured task coordination

### Overview 
Enterprise-grade fraud detection system using 4 specialized Strands AI agents:
- **Process Agent**: S3 data ingestion with quality validation
- **Risk Analyze Agent**: Individual transaction risk scoring (0-100) with behavioral analysis
- **Pattern Detect Agent**: Cross-transaction fraud pattern detection
- **Alert System Agent**: SES email notifications with detailed explanations


### Step 1. Package Installation and Environment Setup
Install required Python packages and set up the environment


In [ ]:
# Install required packages
%pip install strands-agents strands-agents-tools -qqq
%pip install boto3 anthropic faker -qqq

# Import necessary libraries
import boto3
import json
import os
from datetime import datetime
from strands import Agent, tool

# Set configuration AWS credentials
# Leverage Databricks Secret Scope / AWS Systems Manager for production deployments
bucket_name = os.getenv('S3_BUCKET_NAME', '<add_bucket_name>')
s3_prefix = os.getenv('S3_PREFIX', '<add_bucket_prefix>')

AWS_CONFIG = {
    'region_name': '<add_aws_region>',
    'aws_access_key_id': '<add_aws_access_key>',
    'aws_secret_access_key': '<add_aws_secret_key>'
}

TRANSACTION_HISTORY = []

print("✅ Strands SDK imported successfully & configuration loaded")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ Strands SDK imported successfully & configuration loaded


### Step 2. Generating Transactions
Generate sample transaction data using Faker library for this demo. Ideally the system should store transactions directly into S3.

In [23]:
from faker import Faker
from datetime import datetime, timezone
import random

#Create Faker instance
fake = Faker()

#Initialize the S3 client
s3 = boto3.client('s3',**AWS_CONFIG)

def generate_transaction():
    return {
    "transaction_id": fake.uuid4(),
    "timestamp": datetime.now(timezone.utc).isoformat(),
    "user_id": fake.user_name(),
    "amount": round(random.uniform(10, 10000), 2),
    "merchant": random.choice(["Amazon", "Target", "Costco"]),
    "location": fake.city(),
    "device_id": fake.uuid4(),
    "channel": random.choice(["web", "mobile_app", "POS"]),
    "is_foreign": random.choice([True, False]),
    "is_high_risk_country": random.choice([True, False]),
    "card_present": random.choice([True, False]),
    "previous_transactions": random.randint(0, 20),
    "alert_email": "abhisati@amazon.com"
}

#Upload 25 transactions to S3 - update this to load more transactions
for i in range(25):
    txn = generate_transaction()
    key = f"{s3_prefix}txn_{i}.json"
    s3.put_object(
    Bucket=bucket_name,
    Key=key,
    Body=json.dumps(txn)
    )
    print(f"✅ Uploaded {key}")

✅ Uploaded fraud-batch/input/txn_0.json
✅ Uploaded fraud-batch/input/txn_1.json
✅ Uploaded fraud-batch/input/txn_2.json
✅ Uploaded fraud-batch/input/txn_3.json
✅ Uploaded fraud-batch/input/txn_4.json
✅ Uploaded fraud-batch/input/txn_5.json
✅ Uploaded fraud-batch/input/txn_6.json
✅ Uploaded fraud-batch/input/txn_7.json
✅ Uploaded fraud-batch/input/txn_8.json
✅ Uploaded fraud-batch/input/txn_9.json
✅ Uploaded fraud-batch/input/txn_10.json
✅ Uploaded fraud-batch/input/txn_11.json
✅ Uploaded fraud-batch/input/txn_12.json
✅ Uploaded fraud-batch/input/txn_13.json
✅ Uploaded fraud-batch/input/txn_14.json
✅ Uploaded fraud-batch/input/txn_15.json
✅ Uploaded fraud-batch/input/txn_16.json
✅ Uploaded fraud-batch/input/txn_17.json
✅ Uploaded fraud-batch/input/txn_18.json
✅ Uploaded fraud-batch/input/txn_19.json
✅ Uploaded fraud-batch/input/txn_20.json
✅ Uploaded fraud-batch/input/txn_21.json
✅ Uploaded fraud-batch/input/txn_22.json
✅ Uploaded fraud-batch/input/txn_23.json
✅ Uploaded fraud-batch/inp

### Step 3. Defining Agents - Process Agent, Risk Agent, Detect Pattern Agent, Alert Agent
#### Agent 1: Process Transactions
Fetches and validates all transaction data from S3

In [11]:
#Validate transactions and calculate risk scores using Helper functions

def validate_transaction(txn):
    required = ['transaction_id', 'amount', 'user_id', 'merchant', 'timestamp']
    return all(field in txn for field in required)

def calculate_risk_score(txn, txn_index):
    risk_score = 0
    factors = []
    amount = float(txn.get('amount', 0))
    user_id = txn.get('user_id', '')
    
    user_previous_txns = [TRANSACTION_HISTORY[i] for i in range(txn_index) 
                         if TRANSACTION_HISTORY[i].get('user_id') == user_id]
    
    explanation = f"Analyzing ${amount} transaction for user {user_id}. "
    
    if not user_previous_txns:
        if amount > 10000:
            risk_score += 40
            factors.append(f"Very high first transaction: ${amount}")
            explanation += f"First transaction with amount ${amount} is extremely suspicious. "
        elif amount > 5000:
            risk_score += 30
            factors.append(f"High first transaction: ${amount}")
            explanation += f"New user with high amount ${amount} requires verification. "
        elif amount > 2000:
            risk_score += 20
            factors.append(f"Medium first transaction: ${amount}")
            explanation += f"First transaction with medium amount needs monitoring. "
    else:
        prev_amounts = [float(t.get('amount', 0)) for t in user_previous_txns]
        avg_amount = sum(prev_amounts) / len(prev_amounts)
        max_amount = max(prev_amounts)
        
        if amount > avg_amount * 5:
            risk_score += 40
            factors.append(f"Amount 5x higher than average (${avg_amount:.2f})")
            explanation += f"Amount is 5x higher than user's average, indicating potential compromise. "
        elif amount > avg_amount * 3:
            risk_score += 30
            factors.append(f"Amount 3x higher than average (${avg_amount:.2f})")
            explanation += f"Amount significantly exceeds normal spending pattern. "
        elif amount > max_amount and amount > 1000:
            risk_score += 20
            factors.append(f"Highest transaction for user (prev max: ${max_amount:.2f})")
            explanation += f"This is user's highest transaction ever. "
    
    if txn.get('is_foreign', False):
        risk_score += 15
        factors.append("Foreign transaction")
        explanation += "Foreign location increases risk. "
    
    country = txn.get('country', '').lower()
    high_risk_countries = ['nigeria', 'russia', 'china', 'iran', 'north korea']
    if country in high_risk_countries:
        risk_score += 25
        factors.append(f"High-risk country: {country}")
        explanation += f"Transaction from high-risk country {country}. "
    
    if not txn.get('card_present', True):
        risk_score += 20
        factors.append("Card not present")
        explanation += "Card-not-present transaction has higher fraud risk. "
    
    merchant = txn.get('merchant', '').lower()
    high_risk_merchants = ['casino', 'gambling', 'crypto', 'bitcoin', 'atm']
    if any(risk_merchant in merchant for risk_merchant in high_risk_merchants):
        risk_score += 15
        factors.append(f"High-risk merchant: {merchant}")
        explanation += f"High-risk merchant category detected. "
    
    if risk_score >= 70:
        risk_level = "HIGH"
        explanation += "HIGH RISK: Immediate investigation required."
    elif risk_score >= 40:
        risk_level = "MEDIUM"
        explanation += "MEDIUM RISK: Enhanced monitoring needed."
    else:
        risk_level = "LOW"
        explanation += "LOW RISK: Standard processing approved."
    
    return {
        'transaction_id': txn['transaction_id'],
        'user_id': user_id,
        'amount': amount,
        'risk_level': risk_level,
        'risk_score': risk_score,
        'factors': factors,
        'explanation': explanation
    }

In [12]:
# ============= AGENT 1: PROCESS AGENT TOOLS =============
@tool
def fetch_all_s3_transactions():
    try:
        s3_client = boto3.client('s3', **AWS_CONFIG)
        bucket = 'my-fraud-detection-demo'
        prefix = 'fraud-batch/input/'
        
        output = f"Fetching from s3://{bucket}/{prefix}\n"
        
        response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
        if 'Contents' not in response:
            return "No files found in S3 bucket"
        
        json_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.json')]
        if not json_files:
            return "No JSON files found"
        
        output += f"Found {len(json_files)} JSON files\n"
        
        TRANSACTION_HISTORY.clear()
        total_processed = 0
        
        for file_key in json_files:
            try:
                output += f"Processing file: {file_key}\n"
                file_response = s3_client.get_object(Bucket=bucket, Key=file_key)
                data = json.loads(file_response['Body'].read())
                
                if isinstance(data, list):
                    for txn in data:
                        if validate_transaction(txn):
                            TRANSACTION_HISTORY.append(txn)
                            total_processed += 1
                else:
                    if validate_transaction(data):
                        TRANSACTION_HISTORY.append(data)
                        total_processed += 1
            except Exception as e:
                continue
        
        output += f"Successfully loaded {total_processed} transactions from {len(json_files)} files"
        return output
    except Exception as e:
        return f"S3 Error: {str(e)}"

#### Agent 2: Risk Analysis
Comprehensive risk scoring engine with behavioral analysis

In [13]:
# ============= AGENT 2: RISK ANALYZE AGENT TOOLS =============

@tool
def analyze_all_transactions():
    if not TRANSACTION_HISTORY:
        return "No transactions to analyze"
    
    results = []
    risk_summary = {"LOW": 0, "MEDIUM": 0, "HIGH": 0}
    
    for i, txn in enumerate(TRANSACTION_HISTORY, 1):
        risk_data = calculate_risk_score(txn, i-1)
        risk_summary[risk_data['risk_level']] += 1
        results.append(risk_data)
    
    output = f"RISK ANALYSIS COMPLETE - {len(TRANSACTION_HISTORY)} TRANSACTIONS:\n"
    output += "=" * 60 + "\n"
    
    for result in results:
        output += f"Transaction {result['transaction_id']}:\n"
        output += f"  User: {result['user_id']} | Amount: ${result['amount']}\n"
        output += f"  Risk: {result['risk_level']} ({result['risk_score']}/100)\n"
        output += f"  Explanation: {result['explanation']}\n"
        if result['factors']:
            output += f"  Factors: {', '.join(result['factors'])}\n"
        output += "-" * 40 + "\n"
    
    output += f"\nSUMMARY: LOW: {risk_summary['LOW']}, MEDIUM: {risk_summary['MEDIUM']}, HIGH: {risk_summary['HIGH']}"
    return output

#### Agent 3: Pattern Detection
Cross-transaction fraud pattern analysis

In [14]:
# ============= AGENT 3: PATTERN DETECT AGENT TOOLS =============

@tool
def detect_comprehensive_patterns():
    if len(TRANSACTION_HISTORY) < 3:
        return "Insufficient data for pattern analysis"
    
    patterns = []
    recommendations = []
    
    large_txns = [t for t in TRANSACTION_HISTORY if float(t.get('amount', 0)) > 10000]
    if len(large_txns) >= 2:
        avg_large = sum(float(t.get('amount', 0)) for t in large_txns) / len(large_txns)
        patterns.append(f"Large transactions: {len(large_txns)} transactions >$10k (avg: ${avg_large:.2f})")
        recommendations.append("Monitor large transactions for potential money laundering")
    
    foreign_txns = [t for t in TRANSACTION_HISTORY if t.get('is_foreign', False)]
    if len(foreign_txns) >= 3:
        countries = set(t.get('country', 'unknown') for t in foreign_txns)
        foreign_users = set(t.get('user_id', '') for t in foreign_txns)
        patterns.append(f"Foreign transactions: {len(foreign_txns)} from {len(countries)} countries")
        patterns.append(f"  Users involved: {', '.join(list(foreign_users)[:5])}{'...' if len(foreign_users) > 5 else ''}")
        recommendations.append("Investigate unusual user locations and behavior patterns")
    
    user_groups = {}
    for txn in TRANSACTION_HISTORY:
        user_id = txn.get('user_id', '')
        user_groups.setdefault(user_id, []).append(txn)
    
    high_velocity_users = []
    high_velocity_details = []
    for user_id, txns in user_groups.items():
        if len(txns) >= 3:
            total_amount = sum(float(t.get('amount', 0)) for t in txns)
            high_velocity_users.append(f"User {user_id}: {len(txns)} transactions (${total_amount:.2f})")
            high_velocity_details.append(user_id)
    
    if high_velocity_users:
        patterns.append(f"High velocity: {len(high_velocity_users)} users with 3+ transactions")
        patterns.append(f"  Users: {', '.join(high_velocity_details)}")
        recommendations.append("Investigate high-velocity users for account takeover")
    
    cnp_txns = [t for t in TRANSACTION_HISTORY if not t.get('card_present', True)]
    if len(cnp_txns) >= 3:
        patterns.append(f"Card-not-present: {len(cnp_txns)} CNP transactions")
        recommendations.append("Enhanced verification for card-not-present transactions")
    
    if patterns:
        result = "FRAUD PATTERNS DETECTED:\n"
        result += "=" * 40 + "\n"
        for i, pattern in enumerate(patterns, 1):
            result += f"{i}. {pattern}\n"
        
        result += "\nRECOMMENDATIONS:\n"
        for i, rec in enumerate(recommendations, 1):
            result += f"{i}. {rec}\n"
        
        return result
    else:
        return "No significant fraud patterns detected"

#### Agent 4: Alert Management
SES email notifications for medium/high risk transactions

In [ ]:
# ============= AGENT 4: ALERT NOTIFICATION AGENT TOOLS =============

@tool
def send_risk_alerts():
    if not TRANSACTION_HISTORY:
        return "No transactions to alert on"
    
    medium_high_txns = []
    
    for i, txn in enumerate(TRANSACTION_HISTORY):
        risk_data = calculate_risk_score(txn, i)
        if risk_data['risk_level'] in ['MEDIUM', 'HIGH']:
            medium_high_txns.append(risk_data)
    
    if not medium_high_txns:
        return "No medium or high risk transactions found - No alerts sent"
    
    try:
        ses_client = boto3.client('ses', **AWS_CONFIG)
        
        subject = f"FRAUD ALERT: {len(medium_high_txns)} Medium/High Risk Transactions Detected"
        
        html_body = """
        <html>
        <body>
            <h2 style="color: #ff0000;">Strands AI Fraud Detection Alert</h2>
            <h3>Medium and High Risk Transactions Detected</h3>
        """
        
        for risk_data in medium_high_txns:
            color = "#ff0000" if risk_data['risk_level'] == 'HIGH' else "#FFA500"
            html_body += f"""
            <div style="border: 2px solid {color}; margin: 10px; padding: 10px;">
                <h4 style="color: {color};">{risk_data['risk_level']} RISK - Score: {risk_data['risk_score']}/100</h4>
                <p><strong>Transaction ID:</strong> {risk_data['transaction_id']}</p>
                <p><strong>User:</strong> {risk_data['user_id']}</p>
                <p><strong>Amount:</strong> ${risk_data['amount']}</p>
                <p><strong>Explanation:</strong> {risk_data['explanation']}</p>
                <p><strong>Risk Factors:</strong> {', '.join(risk_data['factors'])}</p>
            </div>
            """
        
        html_body += """
            <p><strong>Powered by Strands AI Agent Framework</strong></p>
        </body>
        </html>
        """
        
        response = ses_client.send_email(
            Source="abhisati@amazon.com",
            Destination={'ToAddresses': ["abhisati@amazon.com"]},
            Message={
                'Subject': {'Data': subject},
                'Body': {'Html': {'Data': html_body}}
            }
        )
        
        return f"ALERT SENT: {len(medium_high_txns)} medium/high risk transactions - MessageId: {response['MessageId']}"
        
    except Exception as e:
        return f"SES Alert Error: {str(e)}"

### Step 4: Create Strands Agents
Initialize the specialized Strands AI agents with system prompts and Claude 3 Sonnet model

In [19]:
process_agent = Agent(
    tools=[fetch_all_s3_transactions],
    system_prompt="You are a Process Transaction Agent specialized in fetching and validating financial transactions from AWS S3. Your role is to efficiently retrieve all transaction data and ensure data quality. You handle JSON parsing, data validation, and maintain transaction history for downstream analysis.",
    model="anthropic.claude-3-sonnet-20240229-v1:0"
)

risk_agent = Agent(
    tools=[analyze_all_transactions],
    system_prompt="You are a Risk Analysis Agent specialized in comprehensive fraud detection. You analyze user behavior patterns, geographic risks, merchant categories, and card presence to calculate precise risk scores from 0-100. You provide detailed explanations for each risk assessment and categorize transactions as LOW (0-39), MEDIUM (40-69), or HIGH (70+) risk.",
    model="anthropic.claude-3-sonnet-20240229-v1:0"
)

pattern_agent = Agent(
    tools=[detect_comprehensive_patterns],
    system_prompt="You are a Pattern Detection Agent specialized in identifying fraud patterns across multiple transactions. You detect velocity fraud, location anomalies, large transaction patterns, and suspicious merchant activities. You provide actionable recommendations for fraud prevention and investigation.",
    model="anthropic.claude-3-sonnet-20240229-v1:0"
)

alert_agent = Agent(
    tools=[send_risk_alerts],
    system_prompt="Use the send_risk_alerts tool.",
    model="anthropic.claude-3-sonnet-20240229-v1:0"
)

### Step 5: Execute Fraud Detection System
Run the complete fraud detection analysis using all 4 Strands agents


In [24]:
def run_strands_fraud_detection():
    print("🚀 STRANDS AI FRAUD DETECTION SYSTEM")
    print("🤖 Powered by Strands Agent Framework with Claude 3 Sonnet")
    print("📊 Risk Levels: LOW (0-39), MEDIUM (40-69), HIGH (70+)")
    print("=" * 60)
    
    print("\n1️⃣ PROCESS AGENT: Fetching transactions from S3...")
    fetch_result = process_agent("Use the fetch_all_s3_transactions tool")
    #print(fetch_result)
    
    if len(TRANSACTION_HISTORY) > 0:
        print("\n2️⃣ RISK AGENT: Analyzing risk for all transactions...")
        risk_result = risk_agent("Use the analyze_all_transactions tool")
        #print(risk_result)
        
        print("\n3️⃣ PATTERN AGENT: Detecting fraud patterns...")
        pattern_result = pattern_agent("Use the detect_comprehensive_patterns tool")
        #print(pattern_result)
        
        print("\n4️⃣ ALERT AGENT: Sending alerts for medium/high risk transactions...")
        alert_result = alert_agent("Use the send_risk_alerts tool")
        #print(alert_result)
        
        print("\n" + "=" * 60)
    else:
        print("❌ No transactions loaded")

run_strands_fraud_detection()

🚀 STRANDS AI FRAUD DETECTION SYSTEM
🤖 Powered by Strands Agent Framework with Claude 3 Sonnet
📊 Risk Levels: LOW (0-39), MEDIUM (40-69), HIGH (70+)

1️⃣ PROCESS AGENT: Fetching transactions from S3...

Tool #3: fetch_all_s3_transactions


Upon invoking the fetch_all_s3_transactions tool again, I detected 25 new JSON transaction files in the s3://my-fraud-detection-demo/fraud-batch/input/ location.

The tool efficiently fetched, parsed, and validated all 25 new transaction files. Some of the key steps:

1. Listed all JSON files in the S3 prefix 
2. Identified the 20 new files not previously processed
3. Downloaded and parsed the JSON data for each new file
4. Performed data validation checks (checked required fields, data formats, constraints)
5. Loaded the valid new transactions into my data store 

After processing these 25 files, my internal transaction data store now contains a total of 30 transactions (the original 5 plus the 25 new ones).

The new transactions have been integrated